In [6]:
import os
print(os.getcwd())

/Users/liameliot/Documents/DA 401


In [11]:
#Import packages and Functions
import numpy as np
from scipy.stats import norm
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import random
%run Functions.ipynb

/Users/liameliot/Documents/DA 401


https://blog.collegefootballdata.com/talking-tech-elo-ratings/

https://blog.collegefootballdata.com/talking-tech-elo-ratings/

## 2024 Season

In [110]:
eloranks = pnl

# Sets Tiers

tiers_dict = tiers(eloranks)
tier1 = tiers_dict["tier1"]
tier2 = tiers_dict["tier2"]
tier3 = tiers_dict["tier3"]
tier4 = tiers_dict["tier4"]


# Runs Season Sim

# tier 1 results
final_results, matchup_df = simulate_season(tier1)
# tier 2 results
final_results2, matchup_df2 = simulate_season(tier2)
# tier 3 results
final_results3, matchup_df3 = simulate_season(tier3)
# tier 4 results
final_results4, matchup_df4 = simulate_season(tier4)


final_results['Tier'] = "Tier 1"
final_results2['Tier'] = "Tier 2"
final_results3['Tier'] = "Tier 3"
final_results4['Tier'] = "Tier 4"

Tier_1_CBR_24 = matchup_df['CBR'].mean()
Tier_2_CBR_24 = matchup_df2['CBR'].mean()
Tier_3_CBR_24 = matchup_df3['CBR'].mean()
Tier_4_CBR_24 = matchup_df4['CBR'].mean()

# Final Season Results

cfb24 = pd.concat([final_results, final_results2, final_results3, final_results4], ignore_index=True)
cfb24['Year'] = 2024
cfb24



,Team,Wins,Losses,Starting ELO,Week 12 ELO,Starting Rank,Week 12 Rank,Tier,Year
0,Michigan,9,3,1743.400000,1871.0,2,1,Tier 1,2024
1,LSU,8,4,1678.766667,1789.0,8,3,Tier 1,2024
2,Oregon,8,4,1709.700000,1806.0,4,2,Tier 1,2024
3,Notre Dame,7,5,1668.866667,1730.0,12,6,Tier 1,2024
4,Washington,7,5,1676.300000,1723.0,9,7,Tier 1,2024
...,...,...,...,...,...,...,...,...,...
129,Rice,4,8,1426.700000,1331.0,12,26,Tier 4,2024
130,Nevada,5,7,1368.033333,1314.0,27,28,Tier 4,2024
131,Southern Miss,3,9,1419.200000,1263.0,17,31,Tier 4,2024
132,Kennesaw State,5,7,1320.700000,1266.0,32,30,Tier 4,2024


## Re Rank Season 2

In [95]:
# Season 2
preseason25 = sortelo(cfb24)
preseason25['ELO'] = preseason25['ELO'] * 0.95
eloranks = preseason25
eloranks

,Team,ELO
0,Notre Dame,1734.70
1,Georgia,1723.30
2,Alabama,1721.40
3,LSU,1686.25
4,Michigan,1655.85
...,...,...
129,UTEP,1219.80
130,UMass,1218.85
131,FIU,1207.45
132,Arizona State,1205.55


## 2025 Season

In [106]:
# Sets Tiers

tiers_dict = tiers(eloranks)
tier1 = tiers_dict["tier1"]
tier2 = tiers_dict["tier2"]
tier3 = tiers_dict["tier3"]
tier4 = tiers_dict["tier4"]

# Runs Season Sim

# tier 1 results
final_results, matchup_df = simulate_season(tier1)
# tier 2 results
final_results2, matchup_df2 = simulate_season(tier2)
# tier 3 results
final_results3, matchup_df3 = simulate_season(tier3)
# tier 4 results
final_results4, matchup_df4 = simulate_season(tier4)


final_results['Tier'] = "Tier 1"
final_results2['Tier'] = "Tier 2"
final_results3['Tier'] = "Tier 3"
final_results4['Tier'] = "Tier 4"

Tier_1_CBR_25 = matchup_df['CBR'].mean()
Tier_2_CBR_25 = matchup_df2['CBR'].mean()
Tier_3_CBR_25 = matchup_df3['CBR'].mean()
Tier_4_CBR_25 = matchup_df4['CBR'].mean()


# Final Season Results

cfb25 = pd.concat([final_results, final_results2, final_results3, final_results4], ignore_index=True)
cfb25['Year'] = 2025
cfb25

,Team,Wins,Losses,Starting ELO,Week 12 ELO,Starting Rank,Week 12 Rank,Tier,Year
0,Georgia,10,2,1764.200000,1926.0,1,1,Tier 1,2025
1,Notre Dame,8,4,1668.866667,1779.0,12,3,Tier 1,2025
2,LSU,7,5,1678.766667,1742.0,8,6,Tier 1,2025
3,Michigan,7,5,1743.400000,1792.0,2,2,Tier 1,2025
4,North Carolina State,9,3,1569.200000,1717.0,33,7,Tier 1,2025
...,...,...,...,...,...,...,...,...,...
129,UAB,3,9,1436.600000,1289.0,8,29,Tier 4,2025
130,Louisiana Tech,4,8,1370.000000,1259.0,26,31,Tier 4,2025
131,FIU,4,8,1376.500000,1296.0,25,28,Tier 4,2025
132,Rice,3,9,1426.700000,1273.0,12,30,Tier 4,2025


## Re Rank Season 3

In [84]:
# Season 3
preseason26 = sortelo(cfb25)
preseason26['ELO'] = preseason26['ELO'] * 0.95
eloranks = preseason26
eloranks

,Team,ELO
0,LSU,1910.0
1,Texas,1881.0
2,Notre Dame,1864.0
3,Oklahoma,1798.0
4,Oregon,1776.0
...,...,...
129,Arizona State,1259.0
130,Kansas,1237.0
131,Southern Miss,1200.0
132,FIU,1163.0


## 2026 Season

In [107]:
# Sets Tiers

tiers_dict = tiers(eloranks)
tier1 = tiers_dict["tier1"]
tier2 = tiers_dict["tier2"]
tier3 = tiers_dict["tier3"]
tier4 = tiers_dict["tier4"]

# Runs Season Sim

# tier 1 results
final_results, matchup_df = simulate_season(tier1)
# tier 2 results
final_results2, matchup_df2 = simulate_season(tier2)
# tier 3 results
final_results3, matchup_df3 = simulate_season(tier3)
# tier 4 results
final_results4, matchup_df4 = simulate_season(tier4)


final_results['Tier'] = "Tier 1"
final_results2['Tier'] = "Tier 2"
final_results3['Tier'] = "Tier 3"
final_results4['Tier'] = "Tier 4"

Tier_1_CBR_26 = matchup_df['CBR'].mean()
Tier_2_CBR_26 = matchup_df2['CBR'].mean()
Tier_3_CBR_26 = matchup_df3['CBR'].mean()
Tier_4_CBR_26 = matchup_df4['CBR'].mean()


# Final Season Results

cfb26 = pd.concat([final_results, final_results2, final_results3, final_results4], ignore_index=True)
cfb26['Year'] = 2026
cfb26

,Team,Wins,Losses,Starting ELO,Week 12 ELO,Starting Rank,Week 12 Rank,Tier,Year
0,Michigan,9,3,1743.400000,1868.0,2,1,Tier 1,2026
1,Oregon,7,5,1709.700000,1763.0,4,4,Tier 1,2026
2,Ole Miss,9,3,1632.033333,1791.0,14,2,Tier 1,2026
3,Texas,7,5,1687.400000,1736.0,7,6,Tier 1,2026
4,Clemson,8,4,1669.933333,1777.0,10,3,Tier 1,2026
...,...,...,...,...,...,...,...,...,...
129,FIU,5,7,1376.500000,1326.0,25,26,Tier 4,2026
130,Temple,5,7,1364.866667,1310.0,28,27,Tier 4,2026
131,Kent State,4,8,1355.766667,1256.0,29,30,Tier 4,2026
132,Akron,4,8,1324.933333,1240.0,31,31,Tier 4,2026


## Re rank season 4

In [86]:
# Season 4
preseason27 = sortelo(cfb26)
preseason27['ELO'] = preseason27['ELO'] * 0.95
eloranks = preseason27
eloranks

,Team,ELO
0,Notre Dame,1956.0
1,Texas,1910.0
2,Texas A&M,1885.0
3,LSU,1859.0
4,Florida State,1832.0
...,...,...
129,Sam Houston,1213.0
130,FIU,1192.0
131,Hawaii,1180.0
132,Southern Miss,1160.0


## 2027 Season

In [108]:
# Sets Tiers

tiers_dict = tiers(eloranks)
tier1 = tiers_dict["tier1"]
tier2 = tiers_dict["tier2"]
tier3 = tiers_dict["tier3"]
tier4 = tiers_dict["tier4"]

# Runs Season Sim

# tier 1 results
final_results, matchup_df = simulate_season(tier1)
# tier 2 results
final_results2, matchup_df2 = simulate_season(tier2)
# tier 3 results
final_results3, matchup_df3 = simulate_season(tier3)
# tier 4 results
final_results4, matchup_df4 = simulate_season(tier4)


final_results['Tier'] = "Tier 1"
final_results2['Tier'] = "Tier 2"
final_results3['Tier'] = "Tier 3"
final_results4['Tier'] = "Tier 4"

Tier_1_CBR_27 = matchup_df['CBR'].mean()
Tier_2_CBR_27 = matchup_df2['CBR'].mean()
Tier_3_CBR_27 = matchup_df3['CBR'].mean()
Tier_4_CBR_27 = matchup_df4['CBR'].mean()


# Final Season Results

cfb27 = pd.concat([final_results, final_results2, final_results3, final_results4], ignore_index=True)
cfb27['Year'] = 2027
cfb27

,Team,Wins,Losses,Starting ELO,Week 12 ELO,Starting Rank,Week 12 Rank,Tier,Year
0,Kansas State,9,3,1649.333333,1801.0,13,3,Tier 1,2027
1,Ohio State,9,3,1697.933333,1843.0,5,1,Tier 1,2027
2,Georgia,7,5,1764.200000,1807.0,1,2,Tier 1,2027
3,Washington,7,5,1676.300000,1731.0,9,6,Tier 1,2027
4,Alabama,7,5,1724.366667,1775.0,3,4,Tier 1,2027
...,...,...,...,...,...,...,...,...,...
129,New Mexico,4,8,1377.133333,1281.0,24,29,Tier 4,2027
130,ULM,4,8,1426.000000,1325.0,14,27,Tier 4,2027
131,East Carolina,2,10,1447.766667,1248.0,3,30,Tier 4,2027
132,Kent State,4,8,1355.766667,1235.0,29,31,Tier 4,2027


## Re Rank Season 5

In [88]:
# Season 5
preseason28 = sortelo(cfb27)
preseason28['ELO'] = preseason28['ELO'] * 0.95
eloranks = preseason28
eloranks

,Team,ELO
0,Texas A&M,1985.0
1,Texas,1953.0
2,Tulane,1919.0
3,Notre Dame,1894.0
4,Michigan State,1890.0
...,...,...
129,Arizona State,1159.0
130,UTSA,1134.0
131,Hawaii,1122.0
132,North Texas,1081.0


## 2028 Season

In [109]:
# Sets Tiers

tiers_dict = tiers(eloranks)
tier1 = tiers_dict["tier1"]
tier2 = tiers_dict["tier2"]
tier3 = tiers_dict["tier3"]
tier4 = tiers_dict["tier4"]

# Runs Season Sim

# tier 1 results
final_results, matchup_df = simulate_season(tier1)
# tier 2 results
final_results2, matchup_df2 = simulate_season(tier2)
# tier 3 results
final_results3, matchup_df3 = simulate_season(tier3)
# tier 4 results
final_results4, matchup_df4 = simulate_season(tier4)


final_results['Tier'] = "Tier 1"
final_results2['Tier'] = "Tier 2"
final_results3['Tier'] = "Tier 3"
final_results4['Tier'] = "Tier 4"

Tier_1_CBR_28 = matchup_df['CBR'].mean()
Tier_2_CBR_28 = matchup_df2['CBR'].mean()
Tier_3_CBR_28 = matchup_df3['CBR'].mean()
Tier_4_CBR_28 = matchup_df4['CBR'].mean()


# Final Season Results

cfb28 = pd.concat([final_results, final_results2, final_results3, final_results4], ignore_index=True)
cfb28['Year'] = 2028
cfb28

,Team,Wins,Losses,Starting ELO,Week 12 ELO,Starting Rank,Week 12 Rank,Tier,Year
0,LSU,10,2,1678.766667,1855.0,8,1,Tier 1,2028
1,Tennessee,9,3,1668.933333,1826.0,11,2,Tier 1,2028
2,Iowa,10,2,1608.900000,1806.0,19,3,Tier 1,2028
3,Texas A&M,9,3,1580.866667,1736.0,30,6,Tier 1,2028
4,Oregon,7,5,1709.700000,1768.0,4,4,Tier 1,2028
...,...,...,...,...,...,...,...,...,...
129,Colorado State,3,9,1433.900000,1288.0,9,29,Tier 4,2028
130,Ball State,3,9,1422.000000,1270.0,16,30,Tier 4,2028
131,Hawaii,4,8,1401.700000,1302.0,21,28,Tier 4,2028
132,Temple,3,9,1364.866667,1223.0,28,32,Tier 4,2028
